In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from transformers.wl_graph_kernel_transformer import WLGraphKernelTransformer
import graph_helper
import dataset_helper
import wl
import os

DATASET = 'ling-spam'
GRAPH_TYPE = 'cooccurrence'
SUFFIX = '-single'
SUFFIX = ''
cache_file = dataset_helper.CACHE_PATH + '/dataset_graph_{}_{}{}.npy'.format(GRAPH_TYPE, DATASET, SUFFIX)

print('Using cache_file: {}'.format(cache_file))

if not os.path.exists(cache_file):
    print('Could not find cachefile. Aborting.'.format(cache_file))
else:
    X, Y = dataset_helper.get_dataset(DATASET, use_cached = True, cache_file = cache_file)
    X, Y = np.array(X), np.array(Y)

p = Pipeline([
    ('wl_transformer', WLGraphKernelTransformer()),
    ('clf', sklearn.linear_model.PassiveAggressiveClassifier())
])

param_grid = dict(
    wl_transformer__H= [3],
    wl_transformer__n_jobs= [1],
    clf__n_iter=[500],
    clf__class_weight = ['balanced']
)

cv = GridSearchCV(estimator = p, param_grid=param_grid, cv=3, scoring = 'f1_macro', n_jobs=1, verbose = 11)
gscv_result = cv.fit(X, Y)

gscv_result.best_estimator_, gscv_result.cv_results_

Using cache_file: data/CACHE/dataset_graph_cooccurrence_ling-spam.npy
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] wl_transformer__H=3, clf__class_weight=balanced, clf__n_iter=500, wl_transformer__n_jobs=1 
WLGraphKernelTransformer.fit: len(X)=1928, H=3
WLGraphKernelTransformer.fit: Found empty graph in training set!


KeyboardInterrupt: 